In [1]:
import os
import gdown
import librosa
import argparse
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
from matplotlib.colors import Normalize

import torch
import torch.nn as nn
import torch.nn.functional as F
import random


os.sys.path.append("../")
from synthesizer import Synthesizer
from datasets.text import Language
from melgan.generator import Generator

In [2]:
hp_path = ['../config/global/default.yaml', '../config/vc/default.yaml']

hp_global = OmegaConf.load(hp_path[0])
hp_vc = OmegaConf.load(hp_path[1])
hp = OmegaConf.merge(hp_global, hp_vc)

parser = argparse.ArgumentParser()
parser.add_argument('--config')
hparams = parser.parse_args(['--config', hp_path])

checkpoint = torch.load('../chkpt/vc/cotatron_trained_vc/ff79670epoch=70.ckpt', map_location='cpu')

model = Synthesizer(hparams)#.cuda()
model.load_state_dict(checkpoint['state_dict'])
model.eval()
model.freeze()

lang = Language(hp.data.lang, hp.data.text_cleaners)

In [21]:
with open('../datasets/metadata/estonian_metadata.txt', 'r') as f:
    sentences = f.readlines()

random_input = random.choice(sentences)
#random_input = 'preprocessed_v2/UT-uudised-Mari/Mari/wavs/0001_art_901_lause_18-22k.wav|Kurb öelda, aga Eestis on haiglaid, kus ei tööta psühhiaatreid.|UT-uudised-Mari\n'
random_input

'preprocessed_v2/ERR-uudised-Meelis_Kompus/Meelis_Kompus/20051227-1130ER_uudised_2005_12_27_11_00_5.312_9.687_planned_high_studio-22k.wav|Kell on üksteist. Eesti Raadio uudistega on stuudios Meelis Kompus.|ERR-uudised-Meelis_Kompus\n'

In [22]:
target_id = 'UT-uudised-Mari'
#target_id = 'UT-uudised-Albert'
#target_id = random.choice(hp.data.speakers)
#target_id = 'ERR-uudised-Tonu_Karjatse'
target_audio_sample ='../data/' + [i.split("|")[0] for i in sentences if i.split('|')[2].strip() == target_id][0]
target_id

'UT-uudised-Mari'

In [23]:
text = random_input.split('|')[1]
source_wavpath = '../data/' + random_input.split('|')[0]

text_norm = torch.LongTensor(lang.text_to_sequence(text, hp.data.text_cleaners))
text_norm = text_norm.unsqueeze(0)#.cuda()

wav_source_original, sr = librosa.load(source_wavpath, sr=None, mono=True)
wav_source_original *= (0.99 / np.max(np.abs(wav_source_original)))

wav_target_sample, sr_sample = librosa.load(target_audio_sample, sr=None, mono=True)
wav_target_sample *= (0.99 / np.max(np.abs(wav_target_sample)))

assert sr == hp.audio.sampling_rate
wav_source = torch.from_numpy(wav_source_original).view(1, 1, -1)#.cuda()
mel_source = model.cotatron.audio2mel(wav_source)

target_speaker = torch.LongTensor([hp.data.speakers.index(target_id)])#.cuda()

with torch.no_grad():
    mel_s_t, alignment, residual = model.inference(text_norm, mel_source, target_speaker)
    
melgan = Generator(80)#.cuda()
melgan_ckpt = torch.load('melgan_libritts_g_only.ckpt', map_location='cpu')
melgan.load_state_dict(melgan_ckpt['model_g'])
melgan.eval()

with torch.no_grad():
    audio_s_t = melgan(mel_s_t).squeeze().cpu().detach().numpy()

In [24]:
print("====== Source =======")
print(text)
ipd.Audio(wav_source_original, rate=22050)

====== Source =======
Kell on üksteist. Eesti Raadio uudistega on stuudios Meelis Kompus.


In [25]:
print("====== Target =======")
ipd.Audio(audio_s_t, rate=22050)

====== Target =======


In [26]:
print("====== Target Voice =======")
ipd.Audio(wav_target_sample, rate=22050)

====== Target Voice =======
